# Лабораторная работа 5

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Вариант 24



In [2]:
A = np.array([
    [118.8, -14, -5, -89.1],
    [-14.85, -20, -5, 0],
    [297, 16, 320, 0],
    [0, 6, -30, -36.3]
])
A

array([[ 118.8 ,  -14.  ,   -5.  ,  -89.1 ],
       [ -14.85,  -20.  ,   -5.  ,    0.  ],
       [ 297.  ,   16.  ,  320.  ,    0.  ],
       [   0.  ,    6.  ,  -30.  ,  -36.3 ]])

In [4]:
b = np.array([943, -80.7, 2602.8, 1.1])
b

array([  9.43000000e+02,  -8.07000000e+01,   2.60280000e+03,
         1.10000000e+00])

### Найдем решения с помощью встроенного метода

In [110]:
x_sol = np.linalg.solve(A, b)
x_sol

array([ 6.23545277, -1.19639624,  2.40629021, -2.21672407])

### Преобразуем систему $Ax = b$ в вид, удобный для итераций:

$$x = Bx + c$$

In [36]:
A.shape

(4, 4)

In [70]:
def transform4zeidel(A, b):
    num_vars = A.shape[1]
    
    B = np.empty(A.shape, dtype=float)
    c = np.empty(b.shape)
    
    for i in range(num_vars):
        l = np.hstack([A[i][:i], [0], A[i][i+1:]])
        B[i] = ((-1)*l)/A[i][i]
        c[i] = b[i]/A[i][i]
    return B, c

In [71]:
B, c = transform4zeidel(A, b)

In [72]:
B

array([[-0.        ,  0.11784512,  0.04208754,  0.75      ],
       [-0.7425    ,  0.        , -0.25      ,  0.        ],
       [-0.928125  , -0.05      , -0.        , -0.        ],
       [ 0.        ,  0.16528926, -0.82644628,  0.        ]])

In [73]:
c

array([ 7.93771044,  4.035     ,  8.13375   , -0.03030303])

### Проверим достаточное условие сходимости итерационных методов: $||B||_\infty<1$

In [74]:
np.linalg.norm(B, ord=np.inf)

0.99249999999999994

Достаточное условие сходимости выполняется

### Реализуем метод Зейделя

In [130]:
def zeid(B, c, k, x0):
    # k -- number of iterations
    # x0 -- initial approx
    num_vars = c.shape[0]
    res = x0.astype(float) # Copy array and cast to float
    
    for i in range(k):
        for var in range(num_vars):
            #print("B[{2}] is {0} and res is {1}".format(B[var], res, var))
            res[var] = np.dot(B[var], res) + c[var]
    return res

In [133]:
x0_1 = np.array([1, 2, 3, 4])

In [134]:
zeid_sol = zeid(B, c, 10, x0_1)
zeid_sol

array([ 6.23957948, -1.19744089,  2.40251234, -2.21377453])

### Найдем величину абсолютной погрешности решения методом Зейделя (10 итераций) и решения, полученного с использованием np.linalg.solve

In [113]:
def absolute_vector_error(vec, vec_true):
    # Absolute error of vector vec
    return np.linalg.norm(vec_true - vec, ord=np.inf)/np.linalg.norm(vec_true)

In [118]:
absolute_vector_error(zeid_sol, x_sol)

0.00057776236332402918

### Возьмем другое начальное приближение

In [135]:
x0_2 = np.array([5, 0, 1, -1])

In [136]:
zeid_sol = zeid(B, c, 10, x0_2)
zeid_sol

array([ 6.23645081, -1.19664889,  2.40537654, -2.21601073])

In [132]:
absolute_vector_error(zeid_sol, x_sol)

0.00013973084942861774

Как видим, в первом случае нами был взят вектор начальных приближений $x_{0, 1} = (1, 2, 3, 4)^T$, и было получено решение с величиной абсолютной погрешности $err_1 = 0.000577$.  

Во втором случае мы взяли вектор $x_{0, 2} = (5, 0, 1, -1)^T$ в качестве начального приближения и получили величину абсолютной погрешности $err_2 = 0.000139$.

Рассчитаем, насколько близко стояли значения начальных приближений к идеальному решению, полученному с помощью np.linalg.solve:

In [138]:
# Идеальное решение x_sol
# Приближение в первом случае x0_1
# Приближение во втором случае x0_2
d_1 = np.linalg.norm(x_sol - x0_1, ord=np.inf)
d_2 = np.linalg.norm(x_sol - x0_2, ord=np.inf)
print("В первом случае: {0}\nВо втором случае: {1}".format(d_1, d_2))

В первом случае: 6.216724066974245
Во втором случае: 1.4062902072909207


Как видим, во втором случае мы взяли начальное приближение, стоящее ближе к истинному корню, чем приближение в первом случае. А значит, с учетом того, что алгоритм работал с фиксированным количеством итераций (10 шт), делем вывод, что во втором случае решение должно получиться с большей точностью и меньшей погрешностью. 

В общем, данный вывод соответствует практически полученным результатам.

### Перепишем алгоритм для нахождения решения с наперед заданной точностью $\epsilon$ и подсчетом количества итераций

In [139]:
def converges(x_prev, x_cur, eps):
    return np.linalg.norm(x_prev - x_cur, ord=np.inf) <= eps

In [148]:
def zeid_eps(B, c, eps, x0):
     # k -- number of iterations
    # x0 -- initial approx
    num_vars = c.shape[0]
    res = x0.astype(float) # Copy array and cast to float
    itercount = 0
    
    while True:
        res_prev = res.copy()
        itercount += 1
        for var in range(num_vars):
            res[var] = np.dot(B[var], res) + c[var]
        
        if converges(res_prev, res, EPS):
            break
    return res, itercount

In [152]:
EPS=1e-6
EPSNUMS = np.int(np.abs(np.log10(EPS)))

zeid_sol_eps = zeid_eps(B, c, EPS, x0_1)

print("Получено решение с точностью {0}: {1}\nЗа {2} итераций".format(EPS, np.around(zeid_sol_eps[0], EPSNUMS), zeid_sol_eps[1]))

Получено решение с точностью 1e-06: [ 6.235453 -1.196396  2.40629  -2.216724]
За 22 итераций
